In [1]:
import pandas as pd
import copy
import requests
from tqdm.notebook import tqdm
from tqdm import tqdm
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By          # - 정보 획득
from selenium.webdriver.common.keys import Keys
# from engine_connect import create_engine_connect, db_connection
from sqlalchemy import create_engine
import os
import re
import time
from bs4 import BeautifulSoup
import warnings
import pymysql
import math
warnings.filterwarnings(action='ignore')
from selenium.webdriver.common.action_chains import ActionChains
import json
from lxml import html



import time
import datetime as dt # 사용시 datetime.dt 로 사용 # 현재 시간 사용
from datetime import datetime # 사용시 함수로 바로 사용 #-> 시간 비교

In [2]:
# * 웹 크롤링 동작
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

import time
# ChromeDriver 실행

from selenium.webdriver.chrome.options import Options

# Chrome 브라우저 옵션 생성
chrome_options = Options()

# User-Agent 설정
chrome_options.add_argument("--user-agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36")

# WebDriver 생성
webdriver_manager_directory = ChromeDriverManager().install()

## 데이터 불러오기

In [3]:
df_suggestion = pd.read_excel("dataset/df_suggestion.xlsx", index_col=0)

In [4]:
# 폭주 키워드 데이터프레임
df_suggestion_keyword_rampage = df_suggestion.query('title.str.contains("폭주")')

# 실시간 아레나 키워드 데이터프레임
keywords = ["실시간 아레나", "실레나"]
query_string = ' | '.join([f'title.str.contains("{keyword}")' for keyword in keywords])
df_suggestion_keyword_arena = df_suggestion.query(query_string)

# df_suggestion_keyword_arena = df_suggestion.query('title.str.contains("실레나")')

In [5]:
df_suggestion_keyword_rampage

,title,url,date,like
275,이벤트 코인으로 폭주룬 685개 샀는데요,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2024.06.05.,0
318,개좆버러지같은 폭주겜,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2024.05.26.,0
528,폭주를 '전략적'으로 사용할 수 있게끔 생각해본 방안 (+ 디버프 관련 한 스푼),https://cafe.naver.com/ArticleRead.nhn?clubid=...,2024.03.24.,1
613,점령전 폭주 횟수좀 줄여주세요.;;;,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2024.02.22.,0
745,"모든 pvp컨텐츠 폭주 1턴제한, 역속성 패널티 강화(뎀감15, 뎀증10 프로씩) ...",https://cafe.naver.com/ArticleRead.nhn?clubid=...,2023.12.14.,0
...,...,...,...,...
6520,투스야 폭주가뭐어쨋다고???,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2020.02.08.,0
6603,투스야뭐라고? 폭주확률이뭐?ㅆㅂ,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2020.01.20.,0
6650,개 ㅅㅂ풍신수에 폭주패시브달앗나 시발,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2020.01.11.,0
6651,하 풍신수개ㅆㅂ폭주 씹 시발진짜,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2020.01.11.,0


In [6]:
df_suggestion_keyword_arena

,title,url,date,like
30,실레나 리플레이,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2024.10.07.,1
112,실레나 프리벤 건의,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2024.08.27.,0
131,실레나 즐겨찾기/룬 프리셋을 만들어주세요,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2024.08.16.,1
149,실레나 할때 유형 표시 좀 해주세요,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2024.08.07.,0
198,실레나 룬 버그?,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2024.07.15.,0
...,...,...,...,...
6525,실레나 한탄합니다 운영자님.,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2020.02.07.,0
6701,실레나 보상 형변석 추가좀,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2020.01.04.,0
6702,실레나 보상에 추가해주세요.,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2020.01.03.,0
6705,실레나 점수 개편좀 해주세요,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2020.01.03.,0


## 데이터 크롤링

### 1. 폭주 키워드

In [7]:
# 1

# 인터넷 창 오픈

webdriver_manager_directory = ChromeDriverManager().install()
correct_webdriver_manager_directory = os.path.join(os.path.dirname(webdriver_manager_directory), "chromedriver.exe")
browser = webdriver.Chrome(service=ChromeService(executable_path=correct_webdriver_manager_directory), options=chrome_options)

# 네이버 로그인
browser.get("https://nid.naver.com/nidlogin.login")

In [11]:
df_suggestion_rampage = pd.DataFrame()

index = 0
for x in tqdm(range(index,len(df_suggestion_keyword_rampage))):
    browser.get(f"{df_suggestion_keyword_rampage['url'].iloc[x]}")  

    browser.implicitly_wait(5)
    
    # iframe 제거하는 코드 - iframe에 해당하는 css selector 입력하면 된다.
    browser.switch_to.frame("cafe_main")

    # url
    url = df_suggestion_keyword_rampage['url'].iloc[x]

    browser.implicitly_wait(5)
    
    try:
        
        # 본문 텍스트
        contents = browser.find_elements(By.CSS_SELECTOR, value = "div.se-text > div.se-component-content")
        content = ""
        for y in range(len(contents)):
            content = content + contents[y].text + ""

        if content.strip() == "":
            # 예외적인 페이지가 존재 1
            contents = browser.find_elements(By.CSS_SELECTOR, value = "div.ContentRenderer > div > p")
            content = ""
            for y in range(0,len(contents)-1): # 마지막줄 : 남겨주신 소중한 의견은 매일 확인하여 논의되고 있습니다. 감사합니다 :D -> 이거때문에 -1
                content = content + contents[y].text + ""

            if content.strip() == "":

                # 예외적인 페이지가 존재 2
                contents = browser.find_elements(By.CSS_SELECTOR, value = "div.NHN_Writeform_Main > p")
                content = ""
                for y in range(len(contents)):
                    content = content + contents[y].text + ""
                    
    except:
        print(f"{df_suggestion_keyword_rampage['url'].iloc[x]}")

    print(content)
    series_addition = {"url" : [url], "content" : [content]}
    df_addition = pd.DataFrame(series_addition)
    df_suggestion_rampage = pd.concat([df_suggestion_rampage, df_addition], ignore_index = True)

    time.sleep(1)


  0%|                                                                                                                                  | 0/102 [00:00<?, ?it/s]

폭주 치피 전설만 한번을 안나오는데 확률 주작맞죠?


  1%|█▏                                                                                                                        | 1/102 [00:05<08:34,  5.09s/it]

실레나 좃같네


  2%|██▍                                                                                                                       | 2/102 [00:08<06:42,  4.03s/it]

...

 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋  | 100/102 [09:57<00:19,  9.75s/it]

      남겨주신 소중한 의견은 매일 확인하여 논의되고 있습니다. 감사합니다 :D


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 101/102 [10:07<00:09,  9.82s/it]

 빛팔라스폰받앗냐 존나 잘노네 ㅋ효적 저항 적중시발 내꺼도70인데 ㅋㅋㅅㅂ나도 효저리더인데개줘가튼거 수치가의미가잇냐


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 102/102 [10:17<00:00,  6.05s/it]


### 2. 실레나 키워드

In [12]:
df_suggestion_arena = pd.DataFrame()

index = 0
for x in tqdm(range(index,len(df_suggestion_keyword_arena))):
    browser.get(f"{df_suggestion_keyword_arena['url'].iloc[x]}")  

    browser.implicitly_wait(5)
    
    # iframe 제거하는 코드 - iframe에 해당하는 css selector 입력하면 된다.
    browser.switch_to.frame("cafe_main")

    # url
    url = df_suggestion_keyword_arena['url'].iloc[x]

    browser.implicitly_wait(5)
    
    try:
        
        # 본문 텍스트
        contents = browser.find_elements(By.CSS_SELECTOR, value = "div.se-text > div.se-component-content")
        content = ""
        for y in range(len(contents)):
            content = content + contents[y].text + ""

        if content.strip() == "":
            # 예외적인 페이지가 존재 1
            contents = browser.find_elements(By.CSS_SELECTOR, value = "div.ContentRenderer > div > p")
            content = ""
            for y in range(0,len(contents)-1): # 마지막줄 : 남겨주신 소중한 의견은 매일 확인하여 논의되고 있습니다. 감사합니다 :D -> 이거때문에 -1
                content = content + contents[y].text + ""

            if content.strip() == "":

                # 예외적인 페이지가 존재 2
                contents = browser.find_elements(By.CSS_SELECTOR, value = "div.NHN_Writeform_Main > p")
                content = ""
                for y in range(len(contents)):
                    content = content + contents[y].text + ""
                    
    except:
        print(f"{df_suggestion_keyword_arena['url'].iloc[x]}")

    print(content)
    series_addition = {"url" : [url], "content" : [content]}
    df_addition = pd.DataFrame(series_addition)
    df_suggestion_arena = pd.concat([df_suggestion_arena, df_addition], ignore_index = True)

    time.sleep(1)


  0%|                                                                                                                                  | 0/241 [00:00<?, ?it/s]

1)재생바
특정 장면을 보고 싶을경우 처음부터 다시 봐야 함
2)티어별 검색
각 티어별로 어떤 조합 , 특정몹으로 어떤 조합을 쓰는지 궁금할경우 직접 스크롤 내려가면서 확인 해야함 매우 불편
갓투스 개발자 형들 제발 해줘! ㅋㅋㅋ


  0%|▌                                                                                                                         | 1/241 [00:05<21:02,  5.26s/it]

서머너즈워가 출시된지 10년이 넘었고 출시된 몹들은 1700여개가 넘으며
실레나 내에서 자주 쓰이는 몹만하더라도 얼추 100개는 넘어간다고 생각합니다.
그런데 이 게임은 아직도 랭크게임에 프리벤이 존재하지 않습니다.
스폐셜아레나처럼 상대방을 확인하고 서로 저격벤하고 게임을 시작하는 패치를 원하지는 않습니다.
시간도 많이 잡아먹고 특정몹만 사용하는 사람에게는 불리한 패치가 될수 있으니까요.
그냥 큐돌리기전에 각자 프리벤할 몹을 미리 선택하고 게임이 시작되면 좋을것 같습니다.
특정 하드 카운터몹들의 존재때문에 사용되지 못하는 몹들이 굉장히 많다고 생각하는데
유저들은 그러한 몹들을 벤을 하고 기존에 사용되지 못하던 몹을 기용하면서 훨씬 다양한 덱구성이 나올것이기에
간단한 프리벤 도입은 무성의한 밸패 3-4번 하는것보다 훨씬 더 큰 메타 변화를 가져오며
지금보다 더 전략적이고 재밌는게임을 만들어 줄거라 생각합니다.


  1%|█                                                                                                                         | 2/241 [00:10<21:58,  5.52s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 239/241 [23:41<00:19,  9.89s/it]

 현 실레나 금3인데위로 두단계인 빨2 이기면 7점 얻고밑으로 한단계인 금2 만나면 13점 깍이고 이게 정상입니까?진짜 금2만 만나면 심장이 멈춰요 덱수준이 금2 금3이 차이 많겠어요?금3 빨2가 차이 많겠어요? 도대체 무슨 기준인지 진짜 할맛안나네요 남겨주신 소중한 의견은 매일 확인하여 논의되고 있습니다. 감사합니다 :D


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 240/241 [23:49<00:09,  9.34s/it]

1.물극지,풍하프년들은 옛날부터 버그심했는데 고쳐주시고쫌 매커니즘을 1도모르겠네2.이거 진짜웃자고그런게아니라 진지하게 물발키리 머리색상 파란색으로좀바꿔주세요다른몹들은 자기속성에맞는 머리색,모자색상가지고있는데 왜얘는 노란색인가요? 풍발하고 개성차이가없음3.실레나마감보상 동1~3 형변석 은1~3 탈것인데 탈것 ㄹㅇ진짜 쓸모없자나요 차라리 형변석도 같이주던지 뭡니까저게+실레나하는이유가 저같은 섬토콘트리아아직아빠쪽에있는섬정자같은경우엔 금1달아서 날개오오라 하나만사자고하는사람들이 대다수일껍니다. 은3에 구매할수있게하면좋겠지만 그럼 개나소나다사게될것이고 실레나참여도가 오히려 더떨어질수있으니 별도의 개선부탁드립니다. 추가적으로 오오라or 연성석(이건오바긴한데 갑자기떠오름) 새로운거 추가해서 포인트좀 다양하게쓸수있게개선좀부탁드립니다.쫌 일좀하시죠 저말고도더 구체적인 예시와 분석한글올리시면서 건의하는데듣지도않고   남겨주신 소중한 의견은 매일 확인하여 논의되고 있습니다. 감사합니다 :D


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 241/241 [24:06<00:00,  6.00s/it]


### 엑셀 내보내기

In [13]:
df_suggestion_rampage.to_excel("dataset/df_suggestion_rampage.xlsx")
df_suggestion_arena.to_excel("dataset/df_suggestion_arena.xlsx")